In [1]:
# for plot
import numpy as np
import zipfile as zpf
import os, sys
import glob
sys.path.append('../')
import parameter_setting as pars
import io_control

from matplotlib import pyplot as plt


from mayavi import mlab
mlab.init_notebook()

Notebook initialized with x3d backend.


In [31]:
filedir = '../../results_scinet/isL/'
!eval { 'ls '+ filedir}

All_steps_in_nA1000_nB1000_L40_U5.0_V10.0_chi2.0_isL
All_steps_in_nA1000_nB100_L40_U5.0_V10.0_chi2.0_isL
All_steps_in_nA1000_nB1100_L40_U5.0_V10.0_chi2.0_isL
All_steps_in_nA1000_nB1200_L40_U5.0_V10.0_chi2.0_isL
All_steps_in_nA1000_nB1300_L40_U5.0_V10.0_chi2.0_isL
All_steps_in_nA1000_nB1400_L40_U5.0_V10.0_chi2.0_isL
All_steps_in_nA1000_nB1500_L40_U5.0_V10.0_chi2.0_isL
All_steps_in_nA1000_nB1600_L40_U5.0_V10.0_chi2.0_isL
All_steps_in_nA1000_nB1700_L40_U5.0_V10.0_chi2.0_isL
All_steps_in_nA1000_nB1800_L40_U5.0_V10.0_chi2.0_isL
All_steps_in_nA1000_nB1900_L40_U5.0_V10.0_chi2.0_isL
All_steps_in_nA1000_nB2000_L40_U5.0_V10.0_chi2.0_isL
All_steps_in_nA1000_nB200_L40_U5.0_V10.0_chi2.0_isL
All_steps_in_nA1000_nB300_L40_U5.0_V10.0_chi2.0_isL
All_steps_in_nA1000_nB400_L40_U5.0_V10.0_chi2.0_isL
All_steps_in_nA1000_nB500_L40_U5.0_V10.0_chi2.0_isL
All_steps_in_nA1000_nB600_L40_U5.0_V10.0_chi2.0_isL
All_steps_in_nA1000_nB700_L40_U5.0_V10.0_chi2.0_isL
All_steps_in_nA1000_nB800_L40_U5.0_V10.0_chi2.0_isL
A

In [32]:
#filename = 'All_steps_in_nA100_nB100_L40_U5.0_V10.0_chi2.0_isL'
filename ='All_steps_in_nA1000_nB500_L40_U5.0_V10.0_chi2.0_isL'

Alocs, Blocs, ABlocs, A2Blocs, info, tt = \
    io_control.read_all_snapshots(filedir  + filename)
NA, NB, L, Ub1, Ub2, chi, is_L_rod = info
    
print(info)    

title = 'All_steps_in_nA' + str(NA) + '_nB' + str(NB) + '_L' + str(L) + \
                     '_U' + str(Ub1) + '_V' + str(Ub2) + '_chi' + str(chi) + \
                     '_' + is_L_rod

filedir: ../../results_scinet/isL/All_steps_in_nA1000_nB500_L40_U5.0_V10.0_chi2.0_isL
number of files: 1002
(1000, 500, 40, 5.0, 10.0, 2.0, 'is')


In [33]:
# Anp, Bnp, ABnp, A2Bnp are numpy array in PBC
def track_center(Anp, Bnp, ABnp, A2Bnp):
    coords = np.concatenate( (Anp, Bnp, ABnp, A2Bnp) )

    dist_min = L*np.sqrt(3)
    Allcoords = coords.copy().astype(float)
    xcms = 0
    for i in range(L):
        Allcoords[:,0] = (coords[:,0]-i) % L # reset PBC x_ori
        Allcoords -= np.mean(Allcoords, axis=0) # set center of mass as origin
        dist_temp = np.mean(np.sqrt( np.sum(Allcoords*Allcoords, axis=1) ))
        if dist_temp < dist_min:
            dist_min = dist_temp
            xcms = i                               
    coords[:,0] -= xcms
    
    dist_min = L*np.sqrt(3)
    ycms = 0   
    for i in range(L):
        Allcoords[:,1] = (coords[:,1]-i) % L  # reset PBC x_ori
        Allcoords -= np.mean(Allcoords, axis=0) # set center of mass as origin
        dist_temp = np.mean(np.sqrt( np.sum(Allcoords*Allcoords, axis=1) ))
        if dist_temp < dist_min:
            dist_min = dist_temp 
            ycms = i   
    coords[:,1] -= ycms
    
    dist_min = L*np.sqrt(3)
    zcms = 0   
    for i in range(L):
        Allcoords[:,2] = (coords[:,2]-i) % L # reset PBC x_ori
        Allcoords -= np.mean(Allcoords, axis=0) # set center of mass as origin
        dist_temp = np.mean(np.sqrt( np.sum(Allcoords*Allcoords, axis=1) ))
        if dist_temp < dist_min:
            dist_min = dist_temp 
            zcms = i

    xyzcms = np.array([xcms, ycms, zcms]) #np.array([-2,0,-10])#
    #print(xyzcms )
    return (Anp - xyzcms) % L , (Bnp - xyzcms) % L , (ABnp - xyzcms) % L , (A2Bnp - xyzcms) % L 

def plot_snapshot(Ax, Bx, ABx, A2Bx , fig, angle_elev=30,angle_azim=60, cms=False, xyzshift=[0,0,0]):
    # convert to numpy array with dim = (x, 3)
    Anp   = np.array(Ax) % L
    Bnp   = np.array(Bx) % L  
    ABnp  = np.reshape(ABx, (2*len(ABx),3) ) % L 
    A2Bnp = np.reshape(A2Bx, (3*len(A2Bx),3) ) % L
    
    if cms: # track proper center of mass to show good clustering pictures/animation  
        Anp, Bnp, ABnp, A2Bnp = track_center(Anp, Bnp, ABnp, A2Bnp)

    # PBC
    Aplot   = Anp % L + np.array(xyzshift)*L
    Bplot   = Bnp % L + np.array(xyzshift)*L
    ABplot  = ABnp % L + np.array(xyzshift)*L
    A2Bplot = A2Bnp % L + np.array(xyzshift)*L
    
    mlab.view(azimuth=angle_azim, elevation=angle_elev,figure=fig)
    
    # colors obtained from matplotlib default
    md_hex = plt.rcParams['axes.prop_cycle'].by_key()['color']
    md_rgb = [ tuple( float(int(h.lstrip('#')[i:i+2], 16))/255 for i in (0, 2, 4)) for h in md_hex   ]

    mlab.clf(figure=fig)
    if len(Ax) > 0:
        mlab.points3d(Aplot[:,0],Aplot[:,1],Aplot[:,2],color=md_rgb[0],scale_factor=1)
    if len(Bx) > 0:
        mlab.points3d(Bplot[:,0],Bplot[:,1],Bplot[:,2],color=md_rgb[1],scale_factor=1)
    if len(ABx) > 0:
        mlab.points3d(ABplot[:,0],ABplot[:,1],ABplot[:,2],color=md_rgb[2],scale_factor=1)
    if len(A2Bx) > 0:
        mlab.points3d(A2Bplot[:,0],A2Bplot[:,1],A2Bplot[:,2],color=md_rgb[3], scale_factor=1)
    
    
    
    """
    for xyz in A2B:
        XYZ = np.array(xyz) % L
        plt.plot(XYZ[:,0],XYZ[:,1],XYZ[:,2],'o',color='C3',ms=5)
        if np.max(np.abs(XYZ[0,:]-XYZ[1,:])) == 1:
        plt.plot(XYZ[0:2,0],XYZ[0:2,1],XYZ[0:2,2], '-b', lw=2)
        if np.max(np.abs(XYZ[1,:]-XYZ[2,:])) == 1:
            plt.plot(XYZ[1:,0],XYZ[1:,1],XYZ[1:,2], '-b', lw=2)
    """    

    # camera angles
     
    mlab.view(distance=135,focalpoint=(L/2,L/2,L/2),figure=fig)
        
    #aax.set_xlim([0,L])
    #aax.set_ylim([0,L])
    #aax.set_zlim([0,L])
    
    return 0

In [34]:
t=100
mfig = mlab.figure(size=(1000,1000))
plot_snapshot(Alocs[t], Blocs[t],ABlocs[t], A2Blocs[t], mfig, angle_elev=115,angle_azim=30, cms=True)
#mlab.view( distance=10,focalpoint=(L/2,L/2,L/2))
mlab.outline(extent=[0,L,0,L,0,L], line_width=2)
mlab.savefig('sss.png')

In [20]:
from IPython.display import HTML

import moviepy.editor as mpy
import imageio

ani = plt.figure()
ims = []

tlen = len(Alocs)-1
fps = 20

mfig = mlab.figure()

def make_frame(t):
    t = int(t*fps)
    mlab.clf()
    plot_snapshot(Alocs[t], Blocs[t],ABlocs[t], A2Blocs[t], mfig, angle_elev=115,angle_azim=30, cms=True)
    mlab.outline(extent=[0,L,0,L,0,L], line_width=10)
    return mlab.screenshot(antialiased=True)

anim = mpy.VideoClip(make_frame, duration=tlen/fps)
#anim.write_gif("movie.gif", fps=fps)
anim.write_videofile("movie_A2Bscinet.mp4", fps=fps)

[MoviePy] >>>> Building video movie.mp4
[MoviePy] Writing video movie.mp4


100%|█████████▉| 1000/1001 [05:18<00:00,  3.04it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: movie.mp4 



<Figure size 432x288 with 0 Axes>

In [102]:
mlab.close(fig=mfig)

TypeError: close() got an unexpected keyword argument 'fig'